In [1]:
import sys
import datetime
import matplotlib.pyplot as plt

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")

# Tudat imports for propagation and estimation
from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation, constants
from tudatpy.kernel.numerical_simulation import environment_setup
from tudatpy.kernel.numerical_simulation import propagation_setup
from tudatpy.kernel.numerical_simulation import estimation, estimation_setup
from tudatpy.kernel.numerical_simulation.estimation_setup import observation

# import MPC interface
from tudatpy.data.mpc import BatchMPC
from tudatpy.data.horizons import HorizonsQuery, HorizonsBatch

# other useful modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

from astropy.time import Time

In [2]:
batch = BatchMPC()
batch.get_observations([433])
batch.filter(epoch_start=datetime.datetime(2021, 11, 5), epoch_end=datetime.datetime(2022, 1, 1))
batch_times = (batch.table.epochJ2000secondsTDB.values)

print(batch_times)
print(batch.size)

[6.89362096e+08 6.89362415e+08 6.89362975e+08 6.89363934e+08
 6.89748536e+08 6.89748734e+08 6.89749130e+08 6.89749329e+08
 6.89830133e+08 6.89830733e+08 6.89831333e+08 6.89831930e+08
 6.89832528e+08 6.89833128e+08 6.89833729e+08 6.89834328e+08
 6.89834928e+08 6.89835527e+08 6.89878000e+08 6.89878276e+08
 6.89878830e+08 6.89879863e+08 6.90743650e+08 6.90743929e+08
 6.90744608e+08 6.90746083e+08 6.90915609e+08 6.90915889e+08
 6.90916405e+08 6.90917360e+08 6.90991335e+08 6.90991674e+08
 6.90992014e+08 6.90992353e+08 6.90992693e+08 6.91077648e+08
 6.91077989e+08 6.91078327e+08 6.91078667e+08 6.91079006e+08
 6.91175761e+08 6.91176001e+08 6.91176481e+08 6.91177360e+08
 6.91682480e+08 6.91682863e+08 6.91683246e+08 6.91683629e+08
 6.91684012e+08]
49


In [3]:
venus = HorizonsQuery(
    query_id="299",
    location="@SSB",
    epoch_start=datetime.datetime(2021, 1, 1),
    epoch_end=datetime.datetime(2023, 1, 1),
    epoch_step="1d",
)
eros = HorizonsQuery(
    query_id="433;",
    location="500@399",
    epoch_list=batch_times
)

In [4]:
a = venus.ephemerides()

print(a)

<class 'dict'>
{'start': '2021-01-01 00:00:00.000000', 'stop': '2023-01-01 00:00:00.000000', 'step': '1d'}
{'start': '2020-12-31 23:58:50.816', 'stop': '2022-12-31 23:58:50.816', 'step': '1d'}
{'start': '2020-12-31 23:58:50.816', 'stop': '2022-12-31 23:58:50.816', 'step': '1d'}
{'start': '2021-01-01 00:00:00.000', 'stop': '2023-01-01 00:00:00.000', 'step': '1d'}
{'start': '2021-01-01 00:00:00.000', 'stop': '2023-01-01 00:00:00.000', 'step': '1d'}
 targetname       datetime_str       ...  PABLat epochJ2000secondsTDB
    ---                              ...   deg                       
----------- ------------------------ ... ------- --------------------
Venus (299) 2021-Jan-01 00:00:00.000 ...  1.4249          662731200.0
Venus (299) 2021-Jan-02 00:00:00.000 ...  1.3385          662817600.0
Venus (299) 2021-Jan-03 00:00:00.000 ...  1.2512          662904000.0
Venus (299) 2021-Jan-04 00:00:00.000 ...  1.1629          662990400.0
Venus (299) 2021-Jan-05 00:00:00.000 ...  1.0737          6

In [5]:
epoch = {
    "start": "2021-01-01 00:00:00.000000",
    "stop": "2023-01-01 00:00:00.000000",
    "step": "1d",
}


start = Time(epoch["start"], format="iso", scale="tdb").utc
# start.format = 'tdb'

print(start)

start = start.utc

print(start)

2020-12-31 23:58:50.816
2020-12-31 23:58:50.816


### RADEC comparison

In [6]:
radec_mpc = batch.table.loc[:, ["epochJ2000secondsTDB", "RA", "DEC"]].reset_index(
    drop=True
)
radec_horizons = (
    eros.ephemerides(extra_precision=True, reference_system="ICRF")
    .to_pandas()
    .loc[:, ["epochJ2000secondsTDB", "RA", "DEC"]]
).reset_index(drop=True)
radec_horizons[["RA", "DEC"]] = radec_horizons[["RA", "DEC"]].apply(np.radians)
print(radec_mpc.head())
print(radec_horizons.head())

# print(radec_mpc.head())
# print("\n\n")
# print(radec_horizons.head())

diff = radec_horizons - radec_mpc
diff[["RA_deg", "DEC_deg"]] = diff[["RA", "DEC"]].apply(np.degrees)

print("max diff:")
print(diff.max())
print(diff)

<class 'numpy.ndarray'>
[2459523.727162, 2459523.730857, 2459523.737337, 2459523.748436, 2459528.19985, 2459528.20214, 2459528.20672, 2459528.20902, 2459529.14426, 2459529.1512, 2459529.15815, 2459529.16505, 2459529.17198, 2459529.17892, 2459529.18588, 2459529.19281, 2459529.19976, 2459529.20669, 2459529.698271, 2459529.701473, 2459529.707884, 2459529.719834, 2459539.71737, 2459539.7206, 2459539.72846, 2459539.745534, 2459541.707639, 2459541.710875, 2459541.716854, 2459541.727903, 2459542.5841, 2459542.58802, 2459542.591959, 2459542.59588, 2459542.59981, 2459543.58309, 2459543.58703, 2459543.59095, 2459543.59488, 2459543.59881, 2459544.718661, 2459544.721436, 2459544.726991, 2459544.737158, 2459550.583459, 2459550.58789, 2459550.59233, 2459550.59676, 2459550.60119]
[2459523.7279627235, 2459523.731657724, 2459523.738137724, 2459523.749236724, 2459528.2006507246, 2459528.202940725, 2459528.2075207247, 2459528.2098207246, 2459529.145060725, 2459529.1520007253, 2459529.158950725, 2459529.1